In [ ]:
# Import statements
import azureml.core
from azureml.core import Experiment
from azureml.core import Workspace, Datastore
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import PipelineData
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep, EstimatorStep
from azureml.train.dnn import TensorFlow
from azureml.train.estimator import Estimator
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.environment import Environment
from azureml.core.runconfig import RunConfiguration
import shutil

In [ ]:
# Define some constants
CT_NAME = "ds3v2-clust"           # Name of our compute cluster
VM_SIZE = "STANDARD_DS3_V2"          # Specify the Azure VM for execution of our pipeline
MIN_NODES = 1                    # Min number of compute nodes in cluster
MAX_NODES = 4                    # Max number of compute nodes in cluster

In [ ]:
# Get reference to the workspace
ws = Workspace.from_config("./ws_config.json")

# Set workspace's environment
env = Environment.from_pip_requirements(name = "HIFIS_env", file_path = "./../requirements.txt")
env.register(workspace=ws)
runconfig = RunConfiguration(conda_dependencies=env.python.conda_dependencies)
print(env.python.conda_dependencies.serialize_to_string())

# Move AML ignore file to root folder
aml_ignore_path = shutil.copy('./.amlignore', './../.amlignore') 

In [ ]:
# Get the blob datastore associated with this workspace
blob_store = Datastore(ws, name='hifis_v2_ds')

# Create data references to folders on the blob
data_dr = DataReference(
    datastore=blob_store,
    data_reference_name="data",
    path_on_datastore="data/")
training_logs_dr = DataReference(
    datastore=blob_store,
    data_reference_name="training_logs_data",
    path_on_datastore="logs/training/")
models_dr = DataReference(
    datastore=blob_store,
    data_reference_name="models_data",
    path_on_datastore="models/")

# Set up references to pipeline data (intermediate pipeline storage).
processed_pd = PipelineData(
    "processed_data",
    datastore=blob_store,
    output_name="processed_data",
    output_mode="mount")

In [ ]:
# Set up the compute target for this experiment
try:
    compute_target = AmlCompute(ws, CT_NAME)
    print("Found existing compute target.")
except ComputeTargetException:
    print("Creating new compute target")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=VM_SIZE, min_nodes=1, max_nodes=4)    
    compute_target = ComputeTarget.create(ws, CT_NAME, provisioning_config)  # Create the compute cluster
    
    # Wait for cluster to be provisioned
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20) 
    
print("Azure Machine Learning Compute attached")
print("Compute targets: ", ws.compute_targets)
compute_target = ws.compute_targets[CT_NAME]

In [ ]:
# Define preprocessing step the ML pipeline
step1 = PythonScriptStep(name="preprocess_step",
                         script_name="azure/preprocess_step/preprocess_step.py",
                         arguments=["--datadir", data_dr, "--preprocesseddir", processed_pd],
                         inputs=[data_dr],
                         outputs=[processed_pd],
                         compute_target=compute_target, 
                         source_directory="./../",
                         runconfig=runconfig,
                         allow_reuse=True)

# Define training step in the ML pipeline
est = TensorFlow(source_directory='./../',
                   script_params=None,
                   compute_target=compute_target,
                   entry_script='azure/train_step/train_step.py',
                   pip_packages=['tensorboard', 'pandas', 'dill', 'numpy', 'imblearn', 'matplotlib', 'tqdm', 'scikit-learn',
                                'category_encoders'],
                   use_gpu=False,
                   framework_version='2.0')
step2 = EstimatorStep(name="estimator_train_step", 
                      estimator=est, 
                      estimator_entry_script_arguments=["--preprocesseddir", processed_pd, "--datadir", data_dr,  
                                                        "--traininglogsdir", training_logs_dr, "--modelsdir", models_dr],
                      runconfig_pipeline_params=None, 
                      inputs=[processed_pd, data_dr, training_logs_dr, models_dr], 
                      outputs=[], 
                      compute_target=compute_target)

# Construct the ML pipeline from the steps
steps = [step1, step2]
single_train_pipeline = Pipeline(workspace=ws, steps=steps)
single_train_pipeline.validate()

# Define a new experiment and submit a new pipeline run to the compute target.
experiment = Experiment(workspace=ws, name='MultiTrainExperiment_v1')
train_run = experiment.submit(single_train_pipeline, regenerate_outputs=False)
print("Pipeline is submitted for execution")

# Move AML ignore file back to original folder
aml_ignore_path = shutil.move(aml_ignore_path, './.amlignore') 

In [ ]:
# Wait for the pipeline to finish running.
train_run.wait_for_completion()

# Delete compute cluster to avoid extra charges
compute_target.delete()